In [1]:
# %pip install -e .
# %pip install langchain_neo4j
# %pip install rank-bm25
# %pip install vecs

In [2]:
from dotenv import load_dotenv

assert load_dotenv(override=True)

python-dotenv could not parse statement starting at line 20


In [ ]:
from src.utils.supabase import generate_supabase_conection_string
import urllib
import os
supabase_password = os.environ["SUPABASE_PASSWORD"]
supabase_connection_string = os.environ["SUPABASE_CONNECTION_STRING"]

def generate_supabase_conection_string(base_string, supabase_password, username=None):
    base_string = base_string.replace("[YOUR-PASSWORD]", urllib.parse.quote(supabase_password))

    if username:
        base_string = base_string.replace("[YOUR-USERNAME]", urllib.parse.quote(username))

    return base_string.strip()



supabase_connection_string = generate_supabase_conection_string(supabase_connection_string, supabase_password)


'postgresql://postgres:%27Y0TF%5Elqf%22%7Cvb%7C/2TYM37@db.dwgvcexnrjfprpvptwmq.supabase.co:5432/postgres'

In [ ]:
from services.routes.mem0 import get_relevant_user_memories, add_user_memories, memory
from pprint import pprint
pprint((memory.get_all(user_id="alice")))


messages = [
    {"role": "user", "content": "I'm planning to watch a movie tonight. Any recommendations?"},
    {"role": "assistant", "content": "How about a thriller movies? They can be quite engaging."},
    {"role": "user", "content": "I'm not a big fan of thriller movies but I love sci-fi movies."},
    {"role": "assistant", "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future."}
]

await add_user_memories(user_id='alice', messages=messages)

# await get_relevant_user_memories(message="what do you like", user_id="alice")

In [ ]:
from services.routes.mem0 import get_relevant_user_memories, add_user_memories
from openai import AsyncOpenAI
import os

openai_api_key = os.environ["OPENAI_API_KEY"]
openai_model = os.environ["OPENAI_MODEL"]
openai_client = AsyncOpenAI(api_key=openai_api_key)


async def chat_with_memories(user_message, user_id="alice", top_k=5):
    relevant_memories = await get_relevant_user_memories(user_message, user_id=user_id, top_k=top_k)

    print(relevant_memories, type(relevant_memories))

    memories_str = (
        "\n".join(f" - {entry.get('memory')}" for entry in relevant_memories['results'] if entry)
        if relevant_memories
        else "No relevant memories found."
    )
    system_prompt = f"""you are a helpful assistant. Answer the question using memories provided.
    User Memories: \n{memories_str}"""

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_message}]

    response = await openai_client.chat.completions.create(model=openai_model, messages=messages)
    assistant_response = response.choices[0].message.content.strip()

    messages.append({"role": "assistant", "content": assistant_response})

    await add_user_memories(user_id=user_id, messages=messages)

    return assistant_response

In [ ]:
# while True:
#     user_inputs = input("You: ").strip()
#     if user_inputs.lower() in ["exit", "quit", "q"]:
#         print("Exiting chat...")
#         break
#     print(f"AI: {(await chat_with_memories(user_inputs))}")

In [ ]:
from src.utils.pydantic_agent_generator import generate_supabase_string

# Test the generate_supabase_string function
username = "user@name"
password = "pass#word"
host = "localhost"
database = "testdb"

connection_string = generate_supabase_string(username, password, host, database)
print("Generated connection string:", connection_string)